<a href="https://colab.research.google.com/github/KU81-ComSci-ProjectEnd-GameCard/main/blob/master/Predictor/data/json_unzip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# load data
**https://drive.google.com/drive/folders/1c7MwTdLxnPgvmPbBEfNWa45YAUU53H0l** make sure to load data form this before processing.  

raw_data file so large to push to github

In [1]:
import re
def clear_SearingBlowPlus(lst):
    return [re.sub(r'Searing Blow\+\d+', 'Searing Blow', item) for item in lst]

In [2]:
def upgrade_lst(lst):
  new_lst = list()
  for item in lst:
    if(item == 'Searing Blow'):
      new_lst.append('Searing Blow')
    else:
      new_lst.append(item+'+1')
  return new_lst

In [3]:
def get_ALL_CARD():
  IC_ATTACK_CARDS = ["Strike_R", "Bash", "Anger", "Body Slam", "Clash", "Cleave", "Clothesline", "Headbutt", "Heavy Blade", "Iron Wave", "Perfected Strike", "Pommel Strike", "Sword Boomerang", "Thunderclap", "Twin Strike", "Wild Strike", "Blood for Blood", "Carnage", "Dropkick", "Hemokinesis", "Pummel", "Rampage", "Reckless Charge", "Searing Blow", "Sever Soul", "Uppercut", "Whirlwind", "Bludgeon", "Feed", "Fiend Fire", "Immolate", "Reaper"]
  IC_SKILL_CARDS = ["Defend_R", "Armaments", "Flex", "Havoc", "Shrug It Off", "True Grit", "Warcry", "Battle Trance", "Bloodletting", "Burning Pact", "Disarm", "Dual Wield", "Entrench", "Flame Barrier", "Ghostly Armor", "Infernal Blade", "Intimidate", "Power Through", "Rage", "Second Wind", "Seeing Red", "Sentinel", "Shockwave", "Spot Weakness", "Double Tap", "Exhume", "Impervious", "Limit Break", "Offering"]
  IC_POWER_CARDS = ["Combust", "Dark Embrace", "Evolve", "Feel No Pain", "Fire Breathing", "Inflame", "Metallicize", "Rupture", "Barricade", "Berserk", "Brutality", "Corruption", "Demon Form", "Juggernaut"]
  IRONCLAD_CARDS = IC_ATTACK_CARDS + IC_SKILL_CARDS + IC_POWER_CARDS

  COLORLESS_CARDS = ["Dramatic Entrance", "Flash of Steel", "Mind Blast", "Swift Strike", "HandOfGreed", "Bite", "Expunger", "Ritual Dagger", "Shiv", "Smite", "Through Violence", "Bandage Up", "Blind", "Dark Shackles", "Deep Breath", "Discovery", "Enlightenment", "Finesse", "Forethought", "Good Instincts", "Impatience", "Jack Of All Trades", "Madness", "Panacea", "PanicButton", "Purity", "Trip", "Apotheosis", "Chrysalis", "Master of Strategy", "Metamorphosis", "Secret Technique", "Secret Weapon", "The Bomb", "Thinking Ahead", "Transmutation", "Violence", "Apparition", "Beta", "Insight", "J.A.X.", "Miracle", "Safety", "Magnetism", "Mayhem", "Panache", "Sadistic Nature", "Omega", ]

  CURSE_CARDS = ["Ascender s Bane", "Clumsy", "Curse of the Bell", "Decay", "Doubt", "Injury", "Necronomicurse", "Normality", "Pain", "Parasite", "Pride", "Regret", "Shame", "Writhe"]
  return IRONCLAD_CARDS + COLORLESS_CARDS + CURSE_CARDS + upgrade_lst(IRONCLAD_CARDS) + upgrade_lst(COLORLESS_CARDS)

In [5]:
import pandas as pd
def get_ic_run(json_data):
  df = pd.DataFrame(json_data)
  runInfo = df['event'].apply(pd.Series)
  runInfo['master_deck'] = runInfo['master_deck'].apply(clear_SearingBlowPlus)
  ## รับแค่ run ของ ironclad
  ALL_CARD = get_ALL_CARD()
  ic_run = runInfo[runInfo['master_deck'].apply(lambda x: all(word in ALL_CARD for word in x))]
  ## เกณคือ ต้องผ่านชั้น 50 แล้ว
  return ic_run.loc[ic_run.floor_reached >= 50]

In [11]:
import zipfile
import gzip
import json
from io import BytesIO

def get_quality_run(raw_data, run_num):
    quality_run = pd.DataFrame()
    with zipfile.ZipFile(raw_data, 'r') as zip_ref:
        for gzip_file_name in zip_ref.namelist():
            with zip_ref.open(gzip_file_name) as gzip_file:
                with gzip.GzipFile(fileobj=gzip_file) as decompressed_file:
                    run_info = json.load(decompressed_file)
                    ic_run = get_ic_run(run_info)
                    quality_run = pd.concat([quality_run, ic_run], axis=0, join='outer')
                    quality_run.drop_duplicates(subset=['play_id'])
                    print(len(quality_run))
                    if(len(quality_run) >= run_num):
                        break
    return quality_run

In [12]:
raw_data = 'Monthly_2020_11-20240801T051321Z-001.zip'

In [13]:
quality_run = get_quality_run(raw_data, 2000)

36
71
119
155
202
245
286
334
379
418
460
508
553
594
649
695
726
747
795
826
866
900
939
990
1036
1083
1125
1167
1214
1265
1302
1348
1384
1427
1463
1498
1533
1564
1600
1641
1690
1727
1772
1816
1855
1899
1924
1967
1998
2043


In [16]:
quality_run = quality_run.reset_index()

In [32]:
import os
file_path = os.path.join('processed_data', 'quality_run.json')
quality_run.to_json(file_path)